In [ ]:
# Import required libraries (keeping your existing imports)
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from flask import Flask, jsonify, request
import asyncio
import websockets
import json


#  Health Monitoring System
class HealthMonitor:
    """Predict astronaut vitals and detect anomalies"""

    def __init__(self, timesteps=10, features=7):
        self.timesteps = timesteps
        self.scaler = MinMaxScaler()
        self.model = self._build_model(timesteps, features)

    def _build_model(self, timesteps, features):
        model = Sequential([
            Input(shape=(timesteps, features)),
            LSTM(64, return_sequences=True, kernel_regularizer='l2'),
            Dropout(0.2),
            LSTM(32, kernel_regularizer='l2'),
            Dense(16, activation='relu'),
            Dense(features, activation='linear')
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                      loss='mse',
                      metrics=['mae'])
        return model

    def train(self, data):
        """Train LSTM model on astronaut health and spacecraft environment data"""
        scaled_data = self.scaler.fit_transform(data)
        X, y = [], []
        for i in range(len(scaled_data) - self.timesteps):
            X.append(scaled_data[i:i+self.timesteps])
            y.append(scaled_data[i+self.timesteps])
        X, y = np.array(X), np.array(y)
        self.model.fit(X, y, epochs=20, batch_size=8, verbose=1)

    def predict(self, recent_data):
        """Predict future vitals and detect anomalies"""
        recent_data = self.scaler.transform(recent_data)
        prediction = self.model.predict(recent_data.reshape(1, self.timesteps, -1))
        return self.scaler.inverse_transform(prediction)[0]



#  Health Risk Analyzer
class HealthRiskAnalyzer:
    """Detects abnormal astronaut vitals and spacecraft conditions and issues alerts"""

    def __init__(self, thresholds=None):
        self.thresholds = thresholds or {
            'heart_rate': (50, 100),  # Normal HR range
            'oxygen_level': (90, 100),  # Normal O2 saturation
            'body_temp': (36.5, 37.5),  # Normal body temperature (Celsius)
            'blood_pressure': (90, 120),  # Systolic BP (Diastolic can be added)
            'hydration_level': (40, 60),  # Hydration Level Percentage
            'co2_level': (35, 45),  # CO2 levels in mmHg
            'craft_oxygen_level': (90, 100),  # Spacecraft Oxygen level
            'craft_co2_level': (35, 45),  # Spacecraft CO2 level
            'internal_pressure': (101, 105),  # Spacecraft internal pressure (kPa)
            'temperature': (18, 22)  # Spacecraft temperature (Celsius)
        }

    def analyze(self, vitals):
        alerts = []
        severity = "Normal"

        # Astronaut Health Checks
        if not (self.thresholds['heart_rate'][0] <= vitals[0] <= self.thresholds['heart_rate'][1]):
            severity = "Mild" if 45 <= vitals[0] <= 105 else "Critical"
            alerts.append(f"🚨 Heart rate abnormal ({vitals[0]} BPM) - {severity}")

        if not (self.thresholds['oxygen_level'][0] <= vitals[1] <= self.thresholds['oxygen_level'][1]):
            severity = "Mild" if 85 <= vitals[1] <= 105 else "Critical"
            alerts.append(f"🚨 Oxygen levels critical ({vitals[1]}%) - {severity}")

        if not (self.thresholds['body_temp'][0] <= vitals[2] <= self.thresholds['body_temp'][1]):
            alerts.append(f"🚨 Body temperature abnormal ({vitals[2]}°C)")

        if not (self.thresholds['blood_pressure'][0] <= vitals[3] <= self.thresholds['blood_pressure'][1]):
            alerts.append(f"🚨 Blood pressure abnormal ({vitals[3]} mmHg)")

        if not (self.thresholds['hydration_level'][0] <= vitals[4] <= self.thresholds['hydration_level'][1]):
            alerts.append(f"🚨 Hydration level abnormal ({vitals[4]}%)")

        if not (self.thresholds['co2_level'][0] <= vitals[5] <= self.thresholds['co2_level'][1]):
            alerts.append(f"🚨 CO2 level critical ({vitals[5]} mmHg)")

        # Spacecraft System Checks
        if not (self.thresholds['craft_oxygen_level'][0] <= vitals[6] <= self.thresholds['craft_oxygen_level'][1]):
            alerts.append(f"🚨 Spacecraft Oxygen level abnormal ({vitals[6]}%)")

        if not (self.thresholds['craft_co2_level'][0] <= vitals[7] <= self.thresholds['craft_co2_level'][1]):
            alerts.append(f"🚨 Spacecraft CO2 level abnormal ({vitals[7]} mmHg)")

        if not (self.thresholds['internal_pressure'][0] <= vitals[8] <= self.thresholds['internal_pressure'][1]):
            alerts.append(f"🚨 Spacecraft internal pressure abnormal ({vitals[8]} kPa)")

        if not (self.thresholds['temperature'][0] <= vitals[9] <= self.thresholds['temperature'][1]):
            alerts.append(f"🚨 Spacecraft temperature abnormal ({vitals[9]}°C)")

        return alerts

# Orbital Dashboard Extension
class ExtendedOrbitalDashboard:
    """Visualizes astronaut health alongside orbital risk"""

    def __init__(self):
        self.fig = None

    def update(self, debris_data, satellite_data, vitals, health_alerts):
        self._create_figure()
        self._plot_3d_view(debris_data, satellite_data)
        self._plot_health_status(vitals, health_alerts)
        self.fig.show()

    def _create_figure(self):
        self.fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scatter3d'}, {'type': 'table'}]],
                                 subplot_titles=['Orbital View', 'Astronaut Health & Spacecraft Condition'])

    def _plot_3d_view(self, debris_data, satellite_data):
        self.fig.add_trace(go.Scatter3d(
            x=[d['position'][0] for d in debris_data],
            y=[d['position'][1] for d in debris_data],
            z=[d['position'][2] for d in debris_data],
            mode='markers',
            marker=dict(size=[d['size']*5 for d in debris_data], color='orange', opacity=0.8),
            name='Debris'
        ))

        self.fig.add_trace(go.Scatter3d(
            x=[s['position'][0] for s in satellite_data],
            y=[s['position'][1] for s in satellite_data],
            z=[s['position'][2] for s in satellite_data],
            mode='markers+text',
            marker=dict(size=10, color='blue'),
            text=[s['name'] for s in satellite_data],
            name='Satellites'
        ))

    def _plot_health_status(self, vitals, alerts):
        table_data = [['Metric', 'Value']] + [
            ['Heart Rate (BPM)', f"{vitals[0]:.1f}"],
            ['Oxygen Level (%)', f"{vitals[1]:.1f}"],
            ['Body Temperature (°C)', f"{vitals[2]:.1f}"],
            ['Blood Pressure (mmHg)', f"{vitals[3]:.1f}"],
            ['Hydration Level (%)', f"{vitals[4]:.1f}"],
            ['CO2 Level (mmHg)', f"{vitals[5]:.1f}"],
            ['Spacecraft Oxygen Level (%)', f"{vitals[6]:.1f}"],
            ['Spacecraft CO2 Level (mmHg)', f"{vitals[7]:.1f}"],
            ['Spacecraft Internal Pressure (kPa)', f"{vitals[8]:.1f}"],
            ['Spacecraft Temperature (°C)', f"{vitals[9]:.1f}"]
        ]
        if alerts:
            table_data.append(['⚠️ Alerts', '<br>'.join(alerts)])

        self.fig.add_trace(go.Table(header=dict(values=['Health Metric', 'Reading'], fill_color='lightgrey'),
                                    cells=dict(values=list(zip(*table_data)), fill_color='white')),
                           row=1, col=2)


#  Real-time Data Streaming
async def stream_vitals():
    async with websockets.connect("ws://localhost:8765") as websocket:
        while True:
            vitals = json.loads(await websocket.recv())
            print(f"Received vitals: {vitals}")

app = Flask(__name__)

@app.route('/predict/', methods=['POST'])
def predict_vitals():
    vitals = request.get_json()
    recent_data = np.array(vitals).reshape(1, 5, -1)
    prediction = health_monitor.predict(recent_data)
    return jsonify({"predicted_vitals": prediction.tolist()})

#  MAIN EXECUTION
# ===============
if __name__ == "__main__":

    debris_data = [{'position': [6871e3, 232e3, 489e3], 'velocity': [7.2e3, 0.3e3, 0.1e3], 'size': 0.5}]
    satellite_data = [{'position': [6872e3, 231e3, 490e3], 'velocity': [7.25e3, 0.3e3, 0.15e3], 'name': 'ISS'}]


    health_monitor = HealthMonitor(timesteps=5, features=10)
    health_analyzer = HealthRiskAnalyzer()
    dashboard = ExtendedOrbitalDashboard()


    simulated_data = pd.DataFrame({
        'heart_rate': np.random.uniform(60, 80, 100),
        'oxygen_level': np.random.uniform(95, 99, 100),
        'body_temp': np.random.uniform(36.5, 37.5, 100),
        'blood_pressure': np.random.uniform(90, 120, 100),
        'hydration_level': np.random.uniform(40, 60, 100),
        'co2_level': np.random.uniform(35, 45, 100),
        'craft_oxygen_level': np.random.uniform(90, 100, 100),
        'craft_co2_level': np.random.uniform(35, 45, 100),
        'internal_pressure': np.random.uniform(101, 105, 100),
        'temperature': np.random.uniform(18, 22, 100)
    })
    health_monitor.train(simulated_data.to_numpy())


    recent_vitals = simulated_data[-5:].to_numpy()
    predicted_vitals = health_monitor.predict(recent_vitals)
    health_alerts = health_analyzer.analyze(predicted_vitals)


    dashboard.update(debris_data, satellite_data, predicted_vitals, health_alerts)

    print("\n=== HEALTH STATUS ===")
    for alert in health_alerts:
        print(alert)


    app.run(debug=True)


Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - loss: 1.2727 - mae: 0.4384
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.9930 - mae: 0.3339
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.8054 - mae: 0.2987
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.6605 - mae: 0.2938
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5379 - mae: 0.2775
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.4364 - mae: 0.2637
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.3637 - mae: 0.2607
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3054 - mae: 0.2616
Epoch 9/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2600 - mae: 0.2596
Epoch 10/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2239 - mae: 0.2621
Epoch 11/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1925 - mae: 0.2576
Epoch 12/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1720 - mae: 0.2595
Epoch 13/20
12/12 ━━━━━━━━━━━━━━━━━━━


=== HEALTH STATUS ===
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
